In [1]:
import time

import torch
from dataTreat.readData import LzgdDataset,img_transforms
import numpy as np
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms
import os,cv2
import numpy as np
import tool.visualizationTool as visTool


train_img_dir='../data/trainImgPart'
train_label_dir='../data/trainLabel1Part'

trainDataset = LzgdDataset(train_img_dir,train_label_dir,img_transforms)
trainDataloader = DataLoader(trainDataset, batch_size=1, shuffle=True, num_workers=0, pin_memory=True)
type(trainDataset),len(trainDataset)

valid_img_dir='../data/validImg'
valid_label_dir='../data/validLabel1'

validDataset = LzgdDataset(valid_img_dir,valid_label_dir,img_transforms)
validDataloader = DataLoader(validDataset, batch_size=1, shuffle=True, num_workers=0, pin_memory=True)
print('读取数据完成')
#


读取数据完成


In [48]:
import torch
from torch import nn,optim
import pandas as pd
import numpy as np
from collections import Counter
import tool.tool as tool
from module.module import Module
from module.lzgdModule import Lzgd
from module.FCNModule import FCN
from module.unet import UNet
from indicator import evaluate as eva
from module.unetplusplusModule import NestedUNet
from loss.dice import BCE_DICE_Loss
import cv2


torch.cuda.is_available()
torch.manual_seed(42)
img,label=trainDataloader.dataset[0]
# visTool.showImage(label)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#超参数
lr=1e-2
criterion = nn.BCEWithLogitsLoss().cuda()
model_unet=Module(model=UNet(n_channels=3,n_classes=3).cuda(),criterion=criterion,LR=lr,flag='unet')
optimizer = torch.optim.Adam(params=model_unet.model.parameters(),lr=lr)
model=model_unet.model




epochs = 1
tj=None
pred_list=[]

for epoch in range(epochs):
    img=tool.Dim3to4(img)
    label=tool.Dim3to4(label)
    model.train()
    img=img.to(device=device)
    pred=model(img)
    # pred=pred.sigmoid()
    pred_list.append(pred)
    loss = criterion(pred, label.to(device=device))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # _, labeled = torch.max(label, dim=1)
    # print(tool.getValueCount(_),tool.getValueCount(labeled))
    # print(_.shape,labeled.shape,label.shape)

    _, preded = torch.max(pred, dim=1)
    print(tool.getValueCount(_),tool.getValueCount(preded))
    print(_.shape,preded.shape,pred.shape)
    # print(epoch,loss.data)
    # cv2.imwrite(f'../data/data_pred/{str(epoch)}.png',tool.tensor2numpy(pred.squeeze().permute(1,2,0))*255)

{-0.39059973: 1, -0.37427106: 1, -0.36812294: 1, -0.36383724: 1, -0.35519844: 1, -0.35093027: 1, -0.3505143: 1, -0.34832: 1, -0.34741098: 1, -0.3455368: 1, -0.34451926: 1, -0.33818227: 1, -0.33563143: 1, -0.33213183: 1, -0.32914296: 1, -0.3279314: 1, -0.32704335: 1, -0.32319978: 1, -0.31951296: 1, -0.31932217: 1, -0.31860483: 1, -0.3184694: 1, -0.31479764: 1, -0.3144193: 1, -0.31047493: 1, -0.3078363: 1, -0.30728433: 1, -0.30450174: 1, -0.3026098: 1, -0.29915872: 1, -0.29818222: 1, -0.29106915: 1, -0.28981185: 1, -0.28757504: 1, -0.28660658: 1, -0.28321868: 1, -0.28320986: 1, -0.28307715: 1, -0.28266025: 1, -0.2816568: 1, -0.28115085: 1, -0.27973354: 1, -0.27869487: 1, -0.27705988: 1, -0.2770506: 1, -0.2746591: 1, -0.27325624: 1, -0.27238888: 1, -0.27211002: 1, -0.27138036: 1, -0.27099538: 1, -0.27027: 1, -0.27017593: 1, -0.26984203: 1, -0.2694531: 1, -0.26627797: 1, -0.26586115: 1, -0.26457113: 1, -0.26372662: 1, -0.26320827: 1, -0.26157197: 1, -0.26091802: 1, -0.26072085: 1, -0.26061

In [28]:
path='../data/data_pred'
os.path.exists(path)

# visTool.showImage(pred_list[50])
#
# tool.getValueCount(pred_list[50])
# cv2.imwrite(r'data_pred/50.png',tool.tensor2numpy(pred_list[50])*255)


True